In [1]:
import matplotlib.pyplot as plt
import numpy as np
from base64 import b64encode
import pandas as pd

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from ipyleaflet import *

In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [5]:
!jupyter nbextension enable --py --sys-prefix ipyleaflet

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: ok


In [ ]:
instance_id = '8090226b-e900-4993-bfa2-65f1d959b140'

In [39]:
sh_wms_url = 'https://services.sentinel-hub.com/ogc/wms/' + instance_id + '?showlogo=0&time=2020-10-02/2020-10-02'
m = Map(center=(5, 20), zoom=10)
#m.add_layer(WMSLayer(url=sh_wms_url, layers="FALSE_COLORSENTINEL-2-L1C", tile_size=512))
m.add_layer(WMSLayer(url=sh_wms_url, layers="TRUE_COLORSENTINEL-2-L1C", tile_size=512))
m

Map(center=[5, 20], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [29]:
import os

if not os.path.exists('wind-global.nc'):
    url = 'https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc'
    import requests
    r = requests.get(url)
    wind_data = r.content
    with open('wind-global.nc', 'wb') as f:
        f.write(wind_data)

In [30]:
import xarray as xr
ds = xr.open_dataset('wind-global.nc')
ds

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360)
Coordinates:
  * lat      (lat) float64 90.0 89.0 88.0 87.0 86.0 ... -87.0 -88.0 -89.0 -90.0
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
Data variables:
    u_wind   (lat, lon) float64 ...
    v_wind   (lat, lon) float64 ...
Attributes:
    centerName:      US National Weather Service - NCEP(WMC)
    disciplineName:  Meteorological products
    refTime:         2016-04-30T06:00:00.000Z

In [31]:
#pip install netcdf4

In [32]:
from ipyleaflet.velocity import Velocity
display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
wind = Velocity(
    data=ds, 
    zonal_speed='u_wind', meridional_speed='v_wind', 
    latitude_dimension='lat', longitude_dimension='lon', 
    velocity_scale=0.01, max_velocity=20, 
    display_options=display_options
)
m.add_layer(wind)

In [33]:
instance_id = '8090226b-e900-4993-bfa2-65f1d959b140'

In [35]:
@interact
def location(latitude='-3', longitude='37'):
    
    def info_select(satellite, layer, date):

        layer = layer + satellite
        date = pd.Timestamp(date)
        sh_wms_url = 'https://services.sentinel-hub.com/ogc/wms/' + instance_id + '?showlogo=0&time={}/{}'.format(date,date)
        m.add_layer(WMSLayer(url=sh_wms_url, layers='{}'.format(layer), tile_size=512))
        m.add_layer(Marker(location=(int(latitude), int(longitude))))
        m.center=(int(latitude),int(longitude))

        return m

    _ = interact(info_select, satellite = widgets.Dropdown(options=['SENTINEL-2-L1C', 'LANDSAT-8-L1C']),
                 layer = widgets.Dropdown(options=['FALSE_COLOR', 'TRUE_COLOR']),
                 date = widgets.DatePicker(value=pd.to_datetime('2020-10-01')))
             

interactive(children=(Text(value='-3', description='latitude'), Text(value='37', description='longitude'), Out…

NameError: name 'sh_wms_url' is not defined